In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import shears as sh
import pyreadr
from threadpoolctl import threadpool_limits

In [3]:

threadpool_limits(16)

In [4]:
# adata = sc.read_h5ad("/home/sturm/projects/2020/pircher-scrnaseq-lung/data/20_build_atlas/add_additional_datasets/03_update_annotation/artifacts/full_atlas_merged.h5ad")
# adata = adata[adata.obs["origin"] == "tumor_primary", :]
# sc.pp.subsample(adata, n_obs=20000)
# adata.write_h5ad("data/luca_20k.h5ad")

In [34]:
tcga = pyreadr.read_r(
    "/home/sturm/projects/2020/pircher-scrnaseq-lung/data/13_tcga/for_scissor/nsclc_primary_tumor.rds"
)[None]
adata_tcga = sc.AnnData(tcga.T)

/local/scratch/sturm/ipykernel_13186/1353321048.py:4: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata_tcga = sc.AnnData(tcga.T)


In [35]:
tcga_meta = pd.read_csv(
    "/home/sturm/projects/2020/pircher-scrnaseq-lung/tables/tcga/clinical_data_for_scissor.tsv",
    sep="\t",
).set_index("TCGA_patient_barcode")
adata_tcga.obs = adata_tcga.obs.join(tcga_meta)

In [36]:
adata_sc = sc.read_h5ad("data/luca_20k.h5ad")

In [ ]:
adata_sc_bk, adata_tcga_bk = adata_sc.copy(), adata_tcga.copy()

## Preprocessing

In this section, we are going to 
 * select the most highly variable genes
 * subset bulk and single-cell data to common genes
 * Perform quantile normalization on both single-cell and bulk data

In [8]:
# subset single-cell data to genes in bulk data. Do this *before* highly-variable gene calculation
adata_sc = adata_sc[:, adata_sc.var_names.isin(adata_tcga.var_names)]

In [9]:
sc.pp.highly_variable_genes(adata_sc, n_top_genes=2000, flavor="seurat_v3", subset=True)

/data/scratch/sturm/conda/envs/shears_dev/lib/python3.9/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


In [10]:
# subset bulk data to the highly variable genes
adata_tcga = adata_tcga[:, adata_sc.var_names]

In [23]:
sh.pp.quantile_norm(adata_sc)
sh.pp.quantile_norm(adata_tcga)

The same can be achieved automatically (though less flexibly) by using `func::shears.pp.recipy_shears`:

In [37]:
adata_sc, adata_tcga = sh.pp.recipe_shears(adata_sc_bk, adata_tcga_bk)